In [15]:
#using Pkg
#Pkg.add("XLSX")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


First I am going to explore what the probabilities are on overage that a team would score per second. 

In [1]:
using DataFrames, XLSX

# Load the data
#loop over all files in data_condensed/goals_for_by_strength folder and load them into a dataframe
df_GF = DataFrame()
first_file = true
for file in readdir("data_condensed/goals_for_by_strength")
    data_table = XLSX.readtable("data_condensed/goals_for_by_strength/$file", "Goals For by Strength (since 20")
    temp_df = DataFrame(data_table)
    
    if first_file
        append!(df_GF, temp_df)
        first_file = false
    else
        append!(df_GF, temp_df[2:end, :])  # Skip the header row
    end
end
#what are the headers/ columns of df?
println(names(df_GF))

["Team", "Game Date", "GP", "W", "L", "T", "OT", "P", "P%", "GF", "GA", "GF 5v5", "GF 4v4", "GF 3v3", "GF 6v5", "GF 6v4", "GF 6v3", "GF 5v4", "GF 5v3", "GF 4v3", "GF 3v4", "GF 3v5", "GF 3v6", "GF 4v5", "GF 4v6", "GF 5v6", "PS GF", "GF/GP"]


In [2]:
using DataFrames, XLSX

# Load the data
#loop over all files in data_condensed/goals_for_by_strength folder and load them into a dataframe
df_GA = DataFrame()
first_file = true
for file in readdir("data_condensed/goals_against_by_strength")
    data_table = XLSX.readtable("data_condensed/goals_against_by_strength/$file", "Goals Against by Strength (sinc")
    temp_df = DataFrame(data_table)
    
    if first_file
        append!(df_GA, temp_df)
        first_file = false
    else
        append!(df_GA, temp_df[2:end, :])  # Skip the header row
    end
end
#what are the headers/ columns of df?
println(names(df_GA))

["Team", "Game Date", "GP", "W", "L", "T", "OT", "P", "P%", "GF", "GA", "GA 5v5", "GA 4v4", "GA 3v3", "GA 6v5", "GA 6v4", "GA 6v3", "GA 5v4", "GA 5v3", "GA 4v3", "GA 3v4", "GA 3v5", "GA 3v6", "GA 4v5", "GA 4v6", "GA 5v6", "PS GA", "GA/GP"]


In [3]:
using DataFrames, XLSX

# Load the data
#loop over all files in data_condensed/goals_for_by_strength folder and load them into a dataframe
df_G_by_Period = DataFrame()
first_file = true
for file in readdir("data_condensed/goals_by_period")
    data_table = XLSX.readtable("data_condensed/goals_by_period/$file", "Goals by Period")
    temp_df = DataFrame(data_table)
    
    if first_file
        append!(df_G_by_Period, temp_df)
        first_file = false
    else
        append!(df_G_by_Period, temp_df[2:end, :])  # Skip the header row
    end
end
#what are the headers/ columns of df?
println(names(df_G_by_Period))

["Team", "Game Date", "GP", "W", "L", "T", "OT", "P", "P%", "EV GF", "PP GF", "SH GF", "GF", "GF in P1", "GF in P2", "GF in P3", "GF in OT", "GA", "GA in P1", "GA in P2", "GA in P3", "GA in OT"]


In [10]:
using DataFrames
using Statistics

# Summarize the data - 

# For each team count number of games played. 
unique_teams_GF = unique(df_GF[:, :Team])
unique_teams_GA = unique(df_GA[:, :Team])
unique_teams_G_by_Period = unique(df_G_by_Period[:, :Team])

#create a master list of unique teams
unique_teams = unique(vcat(unique_teams_GF, unique_teams_GA, unique_teams_G_by_Period))
#println(unique_teams)

# Function to clean and convert columns
function clean_and_convert_column!(df::DataFrame, col::Symbol)
    for i in 1:nrow(df)
        if df[i, col] isa String
            if df[i, col] == "--" || df[i, col] == "-" 
                df[i, col] = "0"
            end
            df[i, col] = parse(Int, df[i, col])
        end
    end
end

# Clean dataframes
clean_and_convert_column!(df_GF, Symbol("GF 5v5"))
clean_and_convert_column!(df_GA, Symbol("GA 5v5"))
clean_and_convert_column!(df_G_by_Period, Symbol("GF in P1"))
clean_and_convert_column!(df_G_by_Period, Symbol("GF in P2"))
clean_and_convert_column!(df_G_by_Period, Symbol("GA in P1"))
clean_and_convert_column!(df_G_by_Period, Symbol("GA in P2"))

#create a dictionary Team Name: 5v5 goals for, 5v5 goals against, total P1 & P2 goals for, total P1 & P2 goals against
team_dict = Dict()

for team in unique_teams
    #if the team is not in the df_GF then set team_GF_5V5_avg to 0
    if team ∉ df_GF[:, :Team]
        team_GF_5V5_avg = 0
    else
        team_GF_5V5_avg = mean(df_GF[df_GF[:, :Team] .== team, Symbol("GF 5v5")])
    end
    #if the team is not in the df_GA then set team_GA_5V5_avg to 0
    if team ∉ df_GA[:, :Team]
        team_GA_5V5_avg = 0
    else
        team_GA_5V5_avg = mean(df_GA[df_GA[:, :Team] .== team, Symbol("GA 5v5")])
    end
    #if the team is not in the df_G_by_Period then set team_GF_5V5_avg to 0
    if team ∉ df_G_by_Period[:, :Team]
        avg_GF_P1P2 = 0
        avg_GA_P1P2 = 0
    else
        sum_GF_P1 = sum(df_G_by_Period[df_G_by_Period[:, :Team] .== team, Symbol("GF in P1")])
        sum_GF_P2 = sum(df_G_by_Period[df_G_by_Period[:, :Team] .== team, Symbol("GF in P2")])
        sum_GA_P1 = sum(df_G_by_Period[df_G_by_Period[:, :Team] .== team, Symbol("GA in P1")])
        sum_GA_P2 = sum(df_G_by_Period[df_G_by_Period[:, :Team] .== team, Symbol("GA in P2")])
        sum_game_count_per_team_for_Period_data = size(df_G_by_Period[df_G_by_Period[:, :Team] .== team, :])[1]
        avg_GF_P1P2 = (sum_GF_P1 + sum_GF_P2) / sum_game_count_per_team_for_Period_data
        avg_GA_P1P2 = (sum_GA_P1 + sum_GA_P2) / sum_game_count_per_team_for_Period_data
    end
    team_dict[team] = [team_GF_5V5_avg, team_GA_5V5_avg, avg_GF_P1P2, avg_GA_P1P2]
end

#print the dictionary
println(team_dict)

Dict{Any, Any}("Arizona Coyotes" => [1.6666666666666667, 2.2222222222222223, 1.3333333333333333, 2.2222222222222223], "Minnesota Wild" => [1.25, 1.84375, 1.375, 1.9375], "Florida Panthers" => [1.8307692307692307, 1.8, 1.6, 1.9384615384615385], "Pittsburgh Penguins" => [2.0689655172413794, 2.0689655172413794, 2.086206896551724, 1.706896551724138], "Philadelphia Flyers" => [1.6666666666666667, 2.3333333333333335, 1.6818181818181819, 2.0], "Boston Bruins" => [1.7526881720430108, 1.913978494623656, 1.641304347826087, 1.5434782608695652], "San Jose Sharks" => [1.6857142857142857, 1.8857142857142857, 1.8055555555555556, 1.8888888888888888], "New York Rangers" => [1.7758620689655173, 1.7241379310344827, 1.912280701754386, 1.6140350877192982], "Montréal Canadiens" => [1.4473684210526316, 1.6578947368421053, 1.4473684210526316, 1.4736842105263157], "Los Angeles Kings" => [1.6363636363636365, 2.227272727272727, 1.5, 2.3636363636363638], "Calgary Flames" => [1.4838709677419355, 2.032258064516129,

In [11]:
#now based on the dictionary, we can calculate the average goals for and against for each team

#across all teams, calculate the average goals for and against
total_GF_5v5 = 0
total_GA_5v5 = 0
total_GF_P1P2 = 0
total_GA_P1P2 = 0
for team in keys(team_dict)
    total_GF_5v5 += team_dict[team][1]
    total_GA_5v5 += team_dict[team][2]
    total_GF_P1P2 += team_dict[team][3]
    total_GA_P1P2 += team_dict[team][4]
end

#calculate the average goals for and against
avg_GF_5v5 = total_GF_5v5 / length(keys(team_dict))
avg_GA_5v5 = total_GA_5v5 / length(keys(team_dict))
avg_GF_P1P2 = total_GF_P1P2 / length(keys(team_dict))
avg_GA_P1P2 = total_GA_P1P2 / length(keys(team_dict))

println("Average Goals For 5v5: ", avg_GF_5v5)
println("Average Goals Against 5v5: ", avg_GA_5v5)
println("Average Goals For P1 & P2: ", avg_GF_P1P2)
println("Average Goals Against P1 & P2: ", avg_GA_P1P2)

Average Goals For 5v5: 1.7917059334434984
Average Goals Against 5v5: 1.9344381797158132
Average Goals For P1 & P2: 1.7338622353023387
Average Goals Against P1 & P2: 1.8851091873690209


In [63]:
#convert to probability of scoring a goal per second 
p_scoring_5v5 = avg_GF_5v5 / (60 * 60)
p_conceding_5v5 = avg_GA_5v5 / (60 * 60)
p_scoring_P1P2 = avg_GF_P1P2 / (40 * 60)
p_conceding_P1P2 = avg_GA_P1P2 / (40 * 60)

println("Probability of scoring a goal per second 5v5: ", p_scoring_5v5)
println("Probability of conceding a goal per second 5v5: ", p_conceding_5v5)
println("Probability of scoring a goal per second based on P1 & P2 data: ", p_scoring_P1P2)
println("Probability of conceding a goal per second based on P1 & P2: ", p_conceding_P1P2)

avg_prob_pseudo_score_p12data = (p_scoring_P1P2 + p_conceding_P1P2) / 2
println("Average probability of scoring a goal per second based on P1 & P2 data: ", avg_prob_pseudo_score_p12data)

Probability of scoring a goal per second 5v5: 0.000497696092623194
Probability of conceding a goal per second 5v5: 0.0005373439388099481
Probability of scoring a goal per second based on P1 & P2 data: 0.0007224425980426411
Probability of conceding a goal per second based on P1 & P2: 0.0007854621614037587
Average probability of scoring a goal per second based on P1 & P2 data: 0.0007539523797231999


Note that here the probability of being scored against is higher than scoring even at full strength, for our model we may need to just use the probability of scoring and since we are playing the other team that is our probability of getting scored on in a given second. 

Could really use either the 1st or the 3rd for: 
- $p_{b1} = p_{a1}$: probability that the opponent scores during a given second in 5v5 play (same as the probability that we score on the opponent during a given second in 5v5 play)

Next pulling some of the censored data.
something is censored if:
- there is no goal at the end of the game after the goalie was pulled and then the censored time = time from pulling goalie to end of game 
- the team was scored on after they pulled the goalie and the censored time = time from when they pulled the goalie to when they were scored on 

In [57]:
#first just dump all the data into a dataframe df_goalie_pull_data 

using DataFrames, CSV

# Define the expected columns as symbols
expected_columns = [
    :season, :game_number, :team_name, :date, :pull_period, :pull_time,
    :goal_for_time, :goal_against_time, :game_end_time, :goal_for_timedelta,
    :goal_against_timedelta, :game_end_timedelta
]

# Load the data
df_goalie_pull_data = DataFrame()

# Function to ensure DataFrame has all expected columns
function ensure_columns(df, expected_columns)
    for col in expected_columns
        if !(col in Symbol.(names(df)))
            df[!, col] = Vector{Union{Missing, Any}}(missing, nrow(df))
        end
    end
    return df
end

# Loop over all files in data_condensed/csv/normal_frmt folder and load them into a dataframe
for file in readdir("data_condensed/csv/normal_frmt")
    temp_df = CSV.read("data_condensed/csv/normal_frmt/$file", DataFrame)
    temp_df = ensure_columns(temp_df, expected_columns)
    append!(df_goalie_pull_data, temp_df, promote=true)
end

# Loop over all files in data_condensed/csv/formatted_without_game_end_time folder and load them into a dataframe
for file in readdir("data_condensed/csv/formatted_without_game_end_time")
    temp_df = CSV.read("data_condensed/csv/formatted_without_game_end_time/$file", DataFrame)
    temp_df = ensure_columns(temp_df, expected_columns)
    append!(df_goalie_pull_data, temp_df, promote=true)
end

;

In [45]:
#now that we have the data we can identify 'censored' 'rows' 
#where the goalie is pulled and the game ends before a goal is scored 
#or the goalie is pulled and a goal is conceded before the game ends

#filter to goalie pulled and goal conceded aka goal_agains_time is not missing
#note that the censored time for this is just the value in the goal_against_timedelta column
censored_rows_conceded = df_goalie_pull_data[.!ismissing.(df_goalie_pull_data[:, :goal_against_time]), :]

#filter to goalie pulled and game ends before a goal is scored aka goal_for_time AND goal_against_time are missing
#note that the censored time for this is just the value in the game_end_timedelta column
censored_rows_no_goal = df_goalie_pull_data[ismissing.(df_goalie_pull_data[:, :goal_for_time]), :]
censored_rows_no_goal = censored_rows_no_goal[ismissing.(censored_rows_no_goal[:, :goal_against_time]), :]

#anytime they successfully score a goal after pulling the goalie
non_censored_rows = df_goalie_pull_data[.!ismissing.(df_goalie_pull_data[:, :goal_for_time]), :]
;

In [66]:
function parse_timedelta(timedelta_str::String31)
    # Split the string into days and time parts
    days_part, time_part = split(timedelta_str, " days ")
    
    # Parse the days
    days = parse(Int, days_part)
    
    # Split the time part into hours, minutes, seconds, and microseconds
    hours, minutes, seconds_microseconds = split(time_part, ":")
    seconds, microseconds = split(seconds_microseconds, ".")
    
    # Convert each part to seconds
    total_seconds = days * 86400 + parse(Int, hours) * 3600 + parse(Int, minutes) * 60 + parse(Int, seconds) + parse(Int, microseconds) / 1e9
    
    return total_seconds
end
;

#for censored_rows_conceded just make a quick array of the censored times
#convert goal_against_timedelta to a float of seconds
censored_times_conceded = []
for i in 1:nrow(censored_rows_conceded)
    #convert the timedelta to seconds
    censored_time = parse_timedelta(censored_rows_conceded[i, :goal_against_timedelta])
    push!(censored_times_conceded, censored_time)
end
println("number of seconds until a goal was conceded after goalie pull: ", censored_times_conceded)
avg_time_conceded = mean(censored_times_conceded)
println("average time until a goal was conceded: ", avg_time_conceded)

censored_times_no_goal = []
for i in 1:nrow(censored_rows_no_goal)
    #convert the timedelta to seconds
    censored_time = parse_timedelta(censored_rows_no_goal[i, :game_end_timedelta])
    push!(censored_times_no_goal, censored_time)
end
println("number of seconds from goalie pull to end of game (no team scored): ", censored_times_no_goal)
avg_time_no_goal = mean(censored_times_no_goal)
println("average time goalie was pulled with no goal happening: ", avg_time_no_goal)

# also get the time in seconds a goal was scored after the goalie was pulled for the non-censored rows
non_censored_times_till_score = []
for i in 1:nrow(non_censored_rows)
    #convert the timedelta to seconds
    censored_time = parse_timedelta(non_censored_rows[i, :goal_for_timedelta])
    push!(non_censored_times_till_score, censored_time)
end
println("number of seconds from goalie pull till team successfully scored: ", non_censored_times_till_score)
avg_time_till_score = mean(non_censored_times_till_score)
println("average time until a goal was scored: ", avg_time_till_score)

hazzard_rate_conceded = 1 / avg_time_conceded
hazzard_rate_score = 1 / avg_time_till_score

number of seconds until a goal was conceded after goalie pull: Any[54.0, 19.0, 63.0, 19.5, 28.5, 17.0, 50.0, 22.0, 88.5, 51.0, 10.0, 41.5, 80.0, 157.5, 12.0, 27.0, 19.0, 81.0, 14.5, 25.0, 116.5, 20.0, 10.0, 32.0, 66.0, 34.0, 82.0, 26.5, 9.0, 6.0, 34.5, 94.0, 100.0, 75.5, 45.0, 9.5, 8.5, 7.0, 35.0, 46.0, 14.5, 53.0, 11.0, 103.5, 13.5, 40.0, 5.5, 83.5, 44.0, 80.0, 8.0, 66.0, 71.0, 5.0, 42.5, 15.5, 70.0, 71.0, 74.0, 44.0, 8.0, 52.0, 4.0, 53.0, 33.5, 99.5, 61.5, 43.5, 27.0, 16.5, 22.0, 92.0, 69.0, 49.5, 79.5, 67.0, 69.0, 61.0, 58.0, 23.0, 44.0, 9.0, 59.5, 34.0, 47.5, 44.0, 30.5, 10.5, 52.0, 17.0, 28.0, 52.5, 39.0, 19.5, 18.0, 38.5, 17.0, 76.5, 26.5, 20.0, 65.0, 41.5, 64.0, 81.0, 34.0, 35.5, 13.0, 12.0, 36.0, 40.5, 55.0, 78.0, 21.0, 16.0, 66.5, 82.0, 38.5, 23.0, 34.0, 85.0, 57.0, 56.5, 37.5, 36.0, 30.0, 40.0, 37.5, 11.5, 75.5, 71.0, 14.0, 33.5, 87.0, 7.0, 43.5, 30.5, 35.0, 69.0, 93.5, 13.0, 75.5, 38.0, 74.5, 19.0, 63.0, 68.0, 90.0, 48.0, 38.0, 77.5, 65.5, 25.5, 29.5, 31.0, 55.5, 55.5, 42.0,

0.023325177002041498

next need to determine:
- $p_{b2}$: probability that the opponent scores during a given second in 6v5 play
- $p_{a2}$: probability that we score during a given second in 6v5 play

using the exponential survival model

$e^{-hazzard*t}$ 

hazzard = $e^{b_o + b_1*x1 + b_2*x2}$


In [62]:
#non_censored_times_till_score, censored_times_no_goal, censored_times_conceded
array_TF_success_time = []
#for length non_censored_times_till_score
for i in 1:length(non_censored_times_till_score)
    time = non_censored_times_till_score[i]
    #append (True, time) to array
    push!(array_TF_success_time, (true, time))
end
for j in 1:length(censored_times_conceded)
    time = censored_times_conceded[j]
    #append (False, time) to array
    push!(array_TF_success_time, (false, time))
end
for k in 1:length(censored_times_no_goal)
    time = censored_times_no_goal[k]
    #append (False, time) to array
    push!(array_TF_success_time, (false, time))
end

using DataFrames, CSV
df_to_save = DataFrame(array_TF_success_time, [:Success, :Time])
CSV.write("array_TF_success_time.csv", df_to_save)

"array_TF_success_time.csv"

In [65]:
array_TF_fail_time = []
#for length non_censored_times_till_score
for i in 1:length(non_censored_times_till_score)
    time = non_censored_times_till_score[i]
    #append (True, time) to array
    push!(array_TF_fail_time, (false, time))
end
for j in 1:length(censored_times_conceded)
    time = censored_times_conceded[j]
    #append (False, time) to array
    push!(array_TF_fail_time, (true, time))
end
for k in 1:length(censored_times_no_goal)
    time = censored_times_no_goal[k]
    #append (False, time) to array
    push!(array_TF_fail_time, (false, time))
end

using DataFrames, CSV
df_to_save = DataFrame(array_TF_fail_time, [:Success, :Time])
CSV.write("array_TF_fail_time.csv", df_to_save)

"array_TF_fail_time.csv"